### Number of samples and accuracy

In [1]:
num_samples = 10


In [2]:
good_results = 5


In [3]:
# success_ratio 
good_results/num_samples * 100


50.0

Looks like 50% are good, but is this true?  How many digits of accuracy do we really have?

In [4]:
delta = 1

In [5]:
(good_results + delta)/num_samples * 100

60.0

That's not good.  We lost all digits with a tiny change. Our sample size is too small.  Let's see what happens with 100 samples.

In [6]:
50/100 * 100

50.0

In [7]:
(50 + delta)/100 * 100

51.0

Better.  One extra sample only changes our result in the second digit.

Let's see how many digits we have.  The smallest change we can make to `num_samples` is 1.  For 10 samples, the smallest normalized change is from (1/10) to (2/10), or (1/10). 
In scientific notation, that's just 1e-1.

For 100 samples, it's 1e-2.  And so on.

Note what happens when we take the negated log base 10 of that:


In [8]:
from math import *
print("samples: {:d}".format(num_samples))
- log10(1/(num_samples))



samples: 10


1.0

And with 100 samples:

In [9]:
print("samples: {:d}".format(100))
- log10(1/(100))

samples: 100


2.0

And 1000 samples:

In [10]:
print("samples: {:d}".format(1000))
- log10(1/(1000))

samples: 1000


3.0

By looking at the normalized smallest possible delta -- one sample -- we get the digit precision of our experiment.

In [11]:
print("{:e}".format((50 + 1)/100 * 100))

5.100000e+01


In [12]:
print("{:e}".format((50 + 0)/100 * 100))

5.000000e+01


The `-log10` estimate for 100 samples was 2 digits, and here we have a single-digit change in the least significant digit -- the second.  Good so far.

Trying this with 1000 samples:

In [13]:
print("{:e} {:e}".format((500 + 1)/1000 * 1000, 500))

5.010000e+02 5.000000e+02


We can try a formula for digits

In [14]:
def accd(samples):
    return -log10(1/samples)
    

In [15]:
accd(100)

2.0

In [16]:
accd(1000)

3.0

### Condition number

Maybe digits aren't quite the right measure.  Sensitivity to change may be more appropriate.

Using the condition number $\kappa$ may give a better idea.

$$\kappa = {\delta_{\mathrm{out}} \over \delta_{\mathrm{in}}} $$

We would have to define our functions more precisely to make use of the common definition found at
https://en.wikipedia.org/wiki/Condition_number

Indstead, we will just use the tradidional *problem* condition number, the ratio above

In the following, we look at $\kappa$ via values, then formula, then loop.  
Note that our output is a percentage so we multiply by 100

In [34]:
deli = 1; delo = ((5+deli)/10 - (5/10))*100
delo/deli

9.999999999999998

In [35]:
deli = 1; n = 500; N = 2*n; 
delo = ((n+deli)/N - (n/N))*100
delo/deli

0.10000000000000009

In [42]:
for ex in range(0, 4):
    deli = 1; n = 5*10**ex; N = 2*n; 
    delo = ((n+deli)/N - (n/N))*100
    print("With {:6d} samples, condition number is {:e}".format(N, delo/deli))
    

With     10 samples, condition number is 1.000000e+01
With    100 samples, condition number is 1.000000e+00
With   1000 samples, condition number is 1.000000e-01
With  10000 samples, condition number is 1.000000e-02


We can see that the sensitivity to single result changes decreases markedly as our sample size grows.  No surprise, but this also shows that percentages for sample sizes less than 10 are *misleading* and basically *useless*.

Just use the raw number ratios in those cases, i.e. 5/10, **not** 50%.